In [519]:
import librosa
import numpy as np
import torch

from src.constants import SAMPLE_RATE, WINDOW_LENGTH

HOP_LENGTH = 20
HOP_LENGTH = int(HOP_LENGTH / 1000 * SAMPLE_RATE)
SAMPLE_RATE = 16000

audio, _ = librosa.load("input/j3ko.wav", sr=SAMPLE_RATE)
audio_l = len(audio)


audio = torch.from_numpy(audio).float()
audio_steps = audio_l // HOP_LENGTH + 1

seq_len = 2.55
seq_len = int(seq_len * SAMPLE_RATE)

chunk_duration = 2.55
overlap = 2
chunk_samples = int(chunk_duration * SAMPLE_RATE)
chunk_overlap = int(chunk_duration * SAMPLE_RATE  // overlap)
chunks = []
# print(len(audio))
# print(len(audio) - chunk_samples)
for i in range(0, len(audio) - chunk_samples + 1, chunk_overlap):
    chunk = audio[i:i + chunk_samples]
    print('end =', i + chunk_samples)
    print('padleft = ', i)
    print('padright = ', len(audio)-i-chunk_samples)
    print('chunk shape = ', chunk.shape)
    print('-------')
    chunk = torch.nn.functional.pad(chunk, (i, len(audio)-i-chunk_samples), 'constant', 0)
    chunk = np.pad(chunk, WINDOW_LENGTH // 2, mode="constant")        
    chunks.append(chunk)


datas = []

for chunk in chunks :
    data = []
    for i in range(len(chunk) // seq_len):
        begin_t = i * seq_len
        end_t = begin_t + seq_len + WINDOW_LENGTH
        data.append(
            chunk[begin_t:end_t],
        )
    chunk = chunk[end_t - WINDOW_LENGTH :]
    padding_size = seq_len + WINDOW_LENGTH - len(chunk)
    data.append(
        np.pad(chunk, (0, padding_size), mode='constant'),
    )
    datas.append(data)

datas = torch.from_numpy(np.array(datas)).float()
chunk_n = datas.shape[1]

end = 40800
padleft =  0
padright =  263673
chunk shape =  torch.Size([40800])
-------
end = 61200
padleft =  20400
padright =  243273
chunk shape =  torch.Size([40800])
-------
end = 81600
padleft =  40800
padright =  222873
chunk shape =  torch.Size([40800])
-------
end = 102000
padleft =  61200
padright =  202473
chunk shape =  torch.Size([40800])
-------
end = 122400
padleft =  81600
padright =  182073
chunk shape =  torch.Size([40800])
-------
end = 142800
padleft =  102000
padright =  161673
chunk shape =  torch.Size([40800])
-------
end = 163200
padleft =  122400
padright =  141273
chunk shape =  torch.Size([40800])
-------
end = 183600
padleft =  142800
padright =  120873
chunk shape =  torch.Size([40800])
-------
end = 204000
padleft =  163200
padright =  100473
chunk shape =  torch.Size([40800])
-------
end = 224400
padleft =  183600
padright =  80073
chunk shape =  torch.Size([40800])
-------
end = 244800
padleft =  204000
padright =  59673
chunk shape =  torch.Size([40800])

In [520]:
len(audio) / seq_len

7.462573529411765

In [521]:
len(chunk[end_t - WINDOW_LENGTH :])/seq_len

0.0

In [522]:
(len(audio) - end_t + WINDOW_LENGTH)/seq_len

0.4625735294117647

In [523]:
datas.shape[1]

8

In [524]:
len(chunks)

13

In [525]:
from src.model import E2E


model = torch.load('runs/Pitch_FL6_0/model.pt')
model.eval()
model = model.cuda()

In [526]:
results = []
overlap_results = []
i = 0
for data in datas:
    data = data.cuda()
    with torch.no_grad():
        result = model(data)
        results.append(result[1].cpu().numpy())
    i += 1

results = torch.from_numpy(np.array(results)).float()

In [527]:
results.shape

torch.Size([13, 8, 128, 360])

In [528]:
from src.utils import to_local_average_cents

cents_pred_chunk = []

for result in results:
    print(result.shape)
    cents_pred = to_local_average_cents(result.view(-1,360).cpu().numpy(),None, 0 )
    cents_pred_chunk.append(cents_pred)

torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])


In [529]:
cents_pred_chunk[0].shape[0] / 8

128.0

In [530]:
cents_pred_chunk[0].shape

(1024,)

In [531]:
freq_l // chunk_n

128

In [532]:
freqs = []
freq_l = cents_pred_chunk[0].shape[0]
freq_result = np.zeros(freq_l)
freq_chunk_size = freq_l // chunk_n
overlap_chunk =  int(freq_chunk_size // overlap)
overlap_freqs = []
for i, cents_pred in enumerate(cents_pred_chunk):
    freq = np.array(
        [10 * (2 ** (cent / 1200)) if cent else 0 for cent in cents_pred]
    )
    st = max(0,i*overlap_chunk)
    print(f'{st}: {st+freq_chunk_size}')
    freq_result[st: st+freq_chunk_size] += freq[st: st+freq_chunk_size]
freq_result[overlap_chunk:st+freq_chunk_size-overlap_chunk] = freq_result[overlap_chunk:st+freq_chunk_size-overlap_chunk] / 2

0: 128
64: 192
128: 256
192: 320
256: 384
320: 448
384: 512
448: 576
512: 640
576: 704
640: 768
704: 832
768: 896


In [533]:
freq_result.shape

(1024,)

In [534]:
freq_result[overlap_chunk:-overlap_chunk].shape

(896,)

In [535]:
import pandas as pd

In [536]:
df = pd.DataFrame({"frequency": freq_result})

In [537]:
df.to_csv('f0.csv', index=False)

In [538]:
import pandas as pd
import numpy as np
from scipy.io.wavfile import write

f0 = pd.read_csv("f0.csv")


def nco(fcw, sr):
    phase = 0
    phase_result = []
    for fcw_samp in fcw:
        ph_step = 2 * np.pi * fcw_samp * 1 / sr
        phase += ph_step
        phase_result.append(phase)
    return np.cos(phase_result)


step_size = 20
SAMPLE_RATE = 16000
quantum = (SAMPLE_RATE * step_size) // 1000
# sample_rate = quantum * (1000 // step_size)

fcw = np.zeros(quantum * f0.shape[0])

for i in range(f0.shape[0]):
    fcw[quantum * (i) : quantum * (i + 1)] = f0["frequency"][i]

output = nco(fcw, SAMPLE_RATE)



write("j3k.wav", SAMPLE_RATE, output)